<a href="https://colab.research.google.com/github/rumorArbaz/Quater_2/blob/main/Assignment_05_Gemini_2_0_Video_and_Audio_Analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q google-genai

In [2]:
!pip install gtts -q

In [4]:
!pip install playsound -q

  Preparing metadata (setup.py) ... done


In [20]:
from google.colab import userdata
GEMINI_VADIO_API: str = userdata.get('GEMINI_VADIO_API')
if(GEMINI_VADIO_API):
  print('API KEY FOUND')
else:
  print('API KEY NOT FOUND')


API KEY FOUND


In [21]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
  api_key=GEMINI_VADIO_API
)

model: str = "gemini-2.0-flash-exp"

In [22]:
video_name = "/content/video clip.mp4"

In [24]:
import time

def upload_video(video_name):
  video_file = client.files.upload(path = video_name)
  while video_file.state == "pending":
    time.sleep(10)
    video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f"Vadio processing complete:" + (video_file.uri or ""))

  return video_file

my_video = upload_video(video_name)

Vadio processing complete:https://generativelanguage.googleapis.com/v1beta/files/ey921k7rycgj


In [31]:
from google.genai.types import Content, Part
prompt = """For each scene in this video, generate caption that discribe the scene with spoken text.
Place each caption into an object with timecode of caption vadio.
"""

video = my_video

response = client.models.generate_content(
    model = model,
    contents =[
        Content(
            role= "user",
            parts= [
                Part(text=prompt),
                Part.from_uri(file_uri=video.uri or "", mime_type=video.mime_type or "")
            ]
               )
]
)
print(response.text)

```json
[
  {
    "timecode": "00:00",
    "text": "The video begins with a view of an outdoor space at night, decorated with string lights. Several blurred figures can be seen moving about in the background. The camera seems to be moving."
  },
  {
    "timecode": "00:01",
    "text": "The camera shifts perspective, still outside, with the string lights overhead. People continue to mill about, though slightly more visible."
  },
   {
    "timecode": "00:02",
    "text": "The scene transitions indoors, the camera moving quickly through what appears to be a sleeping area. There are beds with blankets and cushions. The room appears to be lit from inside."
  }
]
```


In [34]:
from gtts import gTTS
from playsound import playsound
from IPython.display import Audio

tts = gTTS(text = "\"hey, What do you know about ceer electric car company."
, lang='en')
tts.save("test.mp3")

with open("output.mp3", "wb") as f:
  for chunk in tts.stream():
    f.write(chunk)

display(Audio("output.mp3", autoplay=True))